#### Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from PIL import Image
from skimage import img_as_ubyte
import tensorflow as tf
import cv2
from keras.preprocessing.image import ImageDataGenerator
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Select data 

In [ ]:
df=pd.read_pickle('/content/drive/MyDrive/Thesis/Dataset/wafer_map.pkl')
df

,waferMap,dieSize,lotName,trianTestLabel,failureType,failureNum,waferMapDim
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
...,...,...,...,...,...,...,...
172945,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2,...",600.0,lot47542,[[Test]],[[Edge-Loc]],2,"(26, 30)"
172946,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47542,[[Test]],[[Edge-Loc]],2,"(26, 30)"
172947,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1,...",600.0,lot47542,[[Test]],[[Edge-Ring]],3,"(26, 30)"
172948,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1,...",600.0,lot47542,[[Test]],[[Edge-Loc]],2,"(26, 30)"


In [ ]:
# Drop fault data
index_fault = []
for i in range(len(df)):
  if np.size(df.waferMap[i],axis=1)< 5 or np.size(df.waferMap[i],axis=0)< 5:
    index_fault.append(df.index[i])

In [ ]:
new_df=df.drop(index_fault,axis=0)

In [ ]:
new_df

,waferMap,dieSize,lotName,trianTestLabel,failureType,failureNum,waferMapDim
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,"(45, 48)"
...,...,...,...,...,...,...,...
172945,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2,...",600.0,lot47542,[[Test]],[[Edge-Loc]],2,"(26, 30)"
172946,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47542,[[Test]],[[Edge-Loc]],2,"(26, 30)"
172947,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1,...",600.0,lot47542,[[Test]],[[Edge-Ring]],3,"(26, 30)"
172948,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1,...",600.0,lot47542,[[Test]],[[Edge-Loc]],2,"(26, 30)"


In [ ]:
faulty_wafer=np.unique(new_df['failureNum'])
for f in faulty_wafer :
    print('{} : {}'.format(f, len(new_df[new_df['failureNum']==f])))

0 : 4294
1 : 555
2 : 5189
3 : 9680
4 : 3593
5 : 866
6 : 1193
7 : 149
8 : 147429


In [ ]:
#Select 300 wafer maps 
number_sample = 300
wafer = []
label = []
for i in range(0,9):
    index_class = np.array(new_df[new_df['failureNum'] == i].index)
    if i != 7:
      for k in range(number_sample):
         f = index_class[k]
         wafer.append(cv2.resize(new_df['waferMap'][f], (224,224), interpolation=cv2.INTER_NEAREST ))
         label.append(i)
    else:     
      for k in range(len(new_df[new_df['failureNum'] == i])):
         f = index_class[k]
         wafer.append(cv2.resize(new_df['waferMap'][f], (224,224), interpolation=cv2.INTER_NEAREST ))
         label.append(i)
wafer = np.array(wafer)
label = np.array(label)

In [ ]:
wafer.shape, label.shape

((2549, 224, 224), (2549,))

In [ ]:
faulty_wafer=np.unique(label)
for f in faulty_wafer :
    print('{} : {}'.format(f, len(label[label==f])))

0 : 300
1 : 300
2 : 300
3 : 300
4 : 300
5 : 300
6 : 300
7 : 149
8 : 300


# Convert to RGB 

In [ ]:
def convert_to_RGB(wafer, image_size):
  wafer_RGB = np.zeros((len(wafer), image_size, image_size, 3))
  for w in range(len(wafer)):
     for i in range(image_size):
        for j in range(image_size):
            wafer_RGB[w, i, j, int(wafer[w, i, j])] = 1
  return wafer_RGB

In [ ]:
wafer_RGB  = convert_to_RGB(wafer, 224)

#Train_test_valid split

In [ ]:
x_train, X_test, y_train, Y_test = train_test_split(wafer_RGB, label, test_size=0.2, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print('x_train: ', X_train.shape)
print('x_test: ',X_test.shape)
print('x_val: ',X_val.shape)
print('y_train: ',Y_train.shape)
print('y_test: ', Y_test.shape)
print('y_test: ', Y_val.shape)

x_train:  (1631, 224, 224, 3)
x_test:  (510, 224, 224, 3)
x_val:  (408, 224, 224, 3)
y_train:  (1631,)
y_test:  (510,)
y_test:  (408,)


In [ ]:
faulty_wafer = np.unique(Y_train)

In [ ]:
for f in faulty_wafer :
    print('{} : {}'.format(f, len(Y_train[Y_train == f])))

0 : 193
1 : 176
2 : 213
3 : 194
4 : 201
5 : 178
6 : 188
7 : 94
8 : 194


In [ ]:
for f in faulty_wafer :
    print('{} : {}'.format(f, len(Y_test[Y_test == f])))

0 : 61
1 : 63
2 : 54
3 : 59
4 : 48
5 : 69
6 : 68
7 : 32
8 : 56


In [ ]:
for f in faulty_wafer :
    print('{} : {}'.format(f, len(Y_val[Y_val == f])))

0 : 46
1 : 61
2 : 33
3 : 47
4 : 51
5 : 53
6 : 44
7 : 23
8 : 50


# Convert array to image and save 

In [ ]:
def save_image(data,label,directory,phase):
  faulty = np.unique(label)
  for f in faulty:
     WM_f = data[label == f]
     for i in range(len(WM_f)):
        img = Image.fromarray(img_as_ubyte(WM_f[i]))
        image_path = (directory + phase + '/' + 'Class_' + str(f) +'/'+ 'WM_class' + str(f) + '_' + str(i) + '.png')
        img.save(image_path)

In [ ]:
directory = '/content/drive/MyDrive/Thesis/Dataset/WM_image/'

In [ ]:
save_image(X_train,Y_train,directory,'Train')

In [ ]:
save_image(X_test,Y_test,directory,'Test')

In [ ]:
save_image(X_val,Y_val,directory,'Valid')